In [1]:
from src.model.filterData import filterData
from src.model.calculateTransformerInformation import calculateTransformerInformation

import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
teste = calculateTransformerInformation(cursor, 'INAT7-01', 'Transformador de Força').executar_calculo()
print(teste)


        CodigoCalculo  ResultadoCalculo UltimaAtualizacaoCalculo  \
0      BCH_CAP_REATOR               1.0  2023-05-30 02:04:36.940   
1      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
2      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
3      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
4      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
...               ...               ...                      ...   
19547  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19548  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19549  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19550  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19551  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   

      descricaoInstalacaoEletrica  idEquipamento  \
0                      Itacaiúnas              1   
1                      Itacaiúnas              1   
2                      Itac

In [6]:
query = '''
SELECT DISTINCT
	   Calculo.Codigo AS CodigoCalculo
	 , ExecucaoCalculoResultado.Resultado AS ResultadoCalculo 
	 , EntradaVariavel.UltimaAtualizacao AS UltimaAtualizacaoCalculo
	 , ie.Descricao AS descricaoInstalacaoEletrica
	 , e.Id AS idEquipamento
	 , e.Descricao AS descricaoEquipamento
	 , Variavel.Codigo As VariavelCodigo
	 , EntradaVariavel.Valor As ValorVariavel
	 , e.FamiliaOnsId
	 , e.TucId
	 , f.Id
	 , f.Abreviatura
	 , f.Nome

FROM [EngineCalculo].[CalculoResultado]  As ExecucaoCalculoResultado
	INNER JOIN [EngineCalculo].[CalculoResultadoEquipamento] 
		ON CalculoResultadoEquipamento.CalculoId = ExecucaoCalculoResultado.CalculoId 
		AND CalculoResultadoEquipamento.JobId = ExecucaoCalculoResultado.JobId
	INNER JOIN [Treetech].[ViewEngine_UltimoJob] 
		ON ViewEngine_UltimoJob.EquipamentoId = CalculoResultadoEquipamento.EquipamentoId 
		AND ViewEngine_UltimoJob.JobId = ExecucaoCalculoResultado.JobId
	INNER JOIN [EngineCalculo].[Calculo] 
		ON Calculo.Id = ExecucaoCalculoResultado.CalculoId
	INNER JOIN [EngineCalculo].[CalculoResultadoVariavel] As EntradaVariavel 
		ON ExecucaoCalculoResultado.JobId = EntradaVariavel.JobId
	INNER JOIN [EngineCalculo].[RevisaoVariavel] As RevisaoVariavel 
		ON RevisaoVariavel.VariavelId = EntradaVariavel.VariavelId 
		AND RevisaoVariavel.Revisao = EntradaVariavel.Revisao
	INNER JOIN [EngineCalculo].[Variavel] 
		ON Variavel.Id = EntradaVariavel.VariavelId 
	INNER JOIN Equipamento AS e 
		ON e.Id = CalculoResultadoEquipamento.EquipamentoId
	INNER JOIN EquipamentoInstalacaoEletrica AS eie 
		ON e.Id = eie.EquipamentoId
	INNER JOIN InstalacaoEletrica AS ie 
		ON  ie.Id = eie.InstalacaoEletricaId
	INNER JOIN Familia AS f 
		ON f.TucId = e.TucId

WHERE 1 = 1
	AND f.Nome = 'Transformador de Força'
	AND e.CodigoOperacional = 'INAT7-01'
	AND Calculo.Codigo IN ('IE_TR_ATV_GP', 'IE_TR_ATV_H2OP', 'REC_MOL_BOLHAS', 'IE_TR_BCH_CAP', 'BCH_CAP_REATOR', 'IE_TR_BCH_TD', 'BCH_TD_REATOR', 'IE_TR_BCH_FUGA')
	-- AND FILTRO PARA O SUBSISTEMA

ORDER BY ie.Descricao,
		 e.Descricao;
'''
cursor.execute(query)
resultado_sql = cursor.fetchall()

colunas = ['CodigoCalculo', 'ResultadoCalculo', 'UltimaAtualizacaoCalculo', 'descricaoInstalacaoEletrica', 'idEquipamento', 'descricaoEquipamento', 'VariavelCodigo', 'ValorVariavel', 'FamiliaOnsId', 'TucId', 'Id', 'Abreviatura', 'Nome']

dados = [dict(zip(colunas, row)) for row in resultado_sql]
df = pd.DataFrame(dados)


In [7]:

print(df)

        CodigoCalculo  ResultadoCalculo UltimaAtualizacaoCalculo  \
0      BCH_CAP_REATOR               1.0  2023-05-30 02:04:36.940   
1      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
2      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
3      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
4      BCH_CAP_REATOR               1.0  2023-05-30 02:04:37.977   
...               ...               ...                      ...   
19547  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19548  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19549  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19550  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   
19551  REC_MOL_BOLHAS               2.0  2023-05-30 02:16:39.557   

      descricaoInstalacaoEletrica  idEquipamento  \
0                      Itacaiúnas              1   
1                      Itacaiúnas              1   
2                      Itac

In [38]:
print(df.idEquipamento.unique())

[ 1  2  3 10]


In [4]:
# Executar a consulta SQL
query = '''
SELECT 
	e.Id
	, e.CodigoOperacional
	, e.TucId
	, e.FamiliaId
	, f.Nome
FROM Equipamento AS e
INNER JOIN EquipamentoInstalacaoEletrica AS eie 
	ON e.Id = eie.EquipamentoId
INNER JOIN InstalacaoEletrica AS ie 
	ON  ie.Id = eie.InstalacaoEletricaId
INNER JOIN Familia AS f 
	ON f.Id = e.FamiliaId
'''
cursor.execute(query)

# Obter os resultados da consulta
resultado_sql = cursor.fetchall()

# Criar o DataFrame
for row in resultado_sql:
    print(row)

(1, 'INAT7-01', 96, 106, 'Autotransformador')
(2, 'INAT7-01', 96, 106, 'Autotransformador')
(3, 'INAT7-01', 96, 106, 'Autotransformador')
(4, 'INAT7-02', 96, 106, 'Autotransformador')
(5, 'INAT7-02', 96, 106, 'Autotransformador')
(6, 'INAT7-02', 96, 106, 'Autotransformador')
(7, 'INAT7-03', 96, 106, 'Autotransformador')
(8, 'INAT7-03', 96, 106, 'Autotransformador')
(9, 'INAT7-03', 96, 106, 'Autotransformador')
(10, 'INAT7-01', 96, 106, 'Autotransformador')
(11, 'ATR-5', 96, 106, 'Autotransformador')
(12, 'ATR-5', 96, 106, 'Autotransformador')
(13, 'ATR-5', 96, 106, 'Autotransformador')
(14, 'ATR-5', 96, 106, 'Autotransformador')
(15, '05T1', 96, 106, 'Autotransformador')
(16, '05T1', 96, 106, 'Autotransformador')
(17, '05T1', 96, 106, 'Autotransformador')
(18, '05T2', 96, 106, 'Autotransformador')
(19, '05T2', 96, 106, 'Autotransformador')
(20, '05T2', 96, 106, 'Autotransformador')
(21, '05T-R', 96, 106, 'Autotransformador')
(22, 'ATR-05T1', 96, 106, 'Autotransformador')
(23, 'ATR-05T1